In [15]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [16]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sess                     = sagemaker.Session()
role                     = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account                  = sess.boto_session.client("sts").get_caller_identity()["Account"]
region                   = sess.boto_session.region_name

In [17]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatglm')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatglm')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-928808346782/chatglm/assets/model.tar.gz


In [18]:
model_name        = None
entry_point       = 'chatglm-inference.py'
framework_version = '1.13.1'
py_version        = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT': '600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [19]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name              = model_name,
    model_data        = model_data,
    entry_point       = entry_point,
    source_dir        = './code',
    role              = role,
    framework_version = framework_version, 
    py_version        = py_version,
    env               = model_environment
)

In [20]:

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name         = None
instance_type         = 'ml.g4dn.2xlarge'
instance_count        = 1

predictor = model.deploy(
    endpoint_name          = endpoint_name,
    instance_type          = instance_type, 
    initial_instance_count = instance_count,
    serializer             = JSONSerializer(),
    deserializer           = JSONDeserializer()
)

-----------!

In [23]:
inputs = {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])


你好！请问有什么需要帮助的吗？


In [21]:
inputs = {
    "ask": "类型#上衣\*材质#牛仔布\*颜色#白色\*风格#简约\*图案#刺绣\*衣样式#外套\*衣款式#破洞"

}

response = predictor.predict(inputs)
print(response["answer"])


1. 上衣：牛仔布颜色为白色，风格简约，图案为刺绣，衣样式为破洞。
2. 外套：材质为牛仔布，颜色为白色，衣款式为宽松直筒设计，搭配破洞元素。
